In [ ]:
##第一章：tensorflow的基本操作

In [ ]:
#第一个案例，构建一个线性模型

In [7]:
#载入数据包
import tensorflow as tf
import numpy as np

In [34]:
#使用numpy生成数据，共100个点

#生成两组数据，每组数据有100项，每个数据在（0，1）
x_data=np.float32(np.random.rand(2,100))
#点乘
#‘1’与x_data的第一个矩阵相乘，‘2’与x_data的第二个矩阵相乘，然后相加生成一个矩阵，在此基础上加上偏移量0.300
y_data=np.dot([1,2],x_data)+0.300

In [35]:
#构建一个线性模型
#tf.zeros([2,3,4]):生成一个三维矩阵，xyz分别为234
#tf.Variable是初始化变量将tf.zeros定义的变量初始化，tensorflow特有的操作，所有的变量都需要分离定义与初始化
b=tf.Variable(tf.zeros([1]))#定义一个一维的一个数据0，并将其初始化给b变量
W=tf.Variable(tf.random_uniform([1,2],-1.0,1.0))#创建一个1x2的矩阵，区间位于（-1.0，1.0）
y=tf.matmul(W,x_data)+b#matmul是矩阵相乘，即线性代数定义的矩阵相乘，如果是multiply则是对应矩阵的对应元素相乘

In [44]:
#最小化方差
#tf.square对矩阵中每一个元素求平方
#tf.reduce_mean计算所有元素的均值
loss=tf.reduce_mean(tf.square(y-y_data))#此时loss得到的是一个数值
#随机梯度下降法，0.5是学习率取值在（0，1）
optimizer=tf.train.GradientDescentOptimizer(0.5)
#将损失函数loss注入optimizer.minimize  loss：即最小化的目标变量，一般就是训练的目标函数，均方差或者交叉熵
train=optimizer.minimize(loss)

In [45]:
#初始化变量
init=tf.initialize_all_variables()
#启动图
sess=tf.Session()
sess.run(init)
#拟合平面
for step in range(0,201):
    sess.run(train)
    if step %20==0:
        print(step,sess.run(W),sess.run(b))
#得到最佳的拟合效果 W:[1，2] b:[0.3]
#即根据数据我们模拟的最好的线性模型为 1*x+2*y+0.3=0




0 [[0.98755556 1.1313674 ]] [1.3532361]
20 [[0.8903092 1.7436365]] [0.475546]
40 [[0.9618708 1.93565  ]] [0.34956113]
60 [[0.9880463 1.9832606]] [0.31394818]
80 [[0.99643284 1.995533  ]] [0.30391786]
100 [[0.9989633 1.998787 ]] [0.30109915]
120 [[0.9997031 1.9996668]] [0.3003082]
140 [[0.9999157 1.9999077]] [0.3000864]
160 [[0.9999762 1.9999743]] [0.30002424]
180 [[0.99999326 1.9999927 ]] [0.30000687]
200 [[0.99999815 1.9999979 ]] [0.3000019]


In [ ]:
#第二个案例，如果构建图

In [2]:
import tensorflow as tf
matrix1=tf.constant([[3.,3.]])
matrix2=tf.constant([[2.],[2.]])
product=tf.matmul(matrix1,matrix2)#矩阵乘法
#此时有三个图，两个constant一个matmul

In [12]:
#在第一个会话中启动图
#启动默认图
sess=tf.Session()
#此时触发了三个图matrix1,matrix2,product
print(sess.run(matrix1))
print(sess.run(matrix2))
result=sess.run(product)
print(result)
#关闭图的会话
sess.close()
#第二种直接启动和关闭图的方法，运行完之后直接关闭
with tf.Session() as sess:
    result=sess.run([product])#[]去掉之后直接显示类型
    print(result)


[[3. 3.]]
[[2.]
 [2.]]
[[12.]]
[array([[12.]], dtype=float32)]


In [ ]:
#第三个案例，如何分配CPU，GPU进行工作

在一套标准的系统上通常有多个计算设备. TensorFlow 支持 CPU 和 GPU 这两种设备. 我们用指定字符串 strings 来标识这些设备. 比如:
"/cpu:0": 机器中的 CPU
"/gpu:0": 机器中的 GPU, 如果你有一个的话.
"/gpu:1": 机器中的第二个 GPU, 以此类推...
如果一个 TensorFlow 的 operation 中兼有 CPU 和 GPU 的实现, 当这个算子被指派设备时, GPU 有优先权. 比如matmul中 CPU 和 GPU kernel 函数都存在. 那么在 cpu:0 和 gpu:0 中, matmul operation 会被指派给 gpu:0 .

In [17]:
#获取操作被指派到哪个设备上运行，使用log_device_placement新建一个session，并设置为True （不知道为什么我的这里没有显示文件内容，同理下面的都未显示，我的IDE是jupyter notebook
a=tf.constant([1.0,2.0,3.0,4.0,5.0,6.0],shape=[2,3],name='a')
b=tf.constant([1.0,2.0,3.0,4.0,5.0,6.0],shape=[3,2],name='b')
c=tf.matmul(a,b)
sess=tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [19]:
#手工指派运行环境CPU
with tf.device('/cpu:0'):
    a=tf.constant([1.0,2.0,3.0,4.0,5.0,6.0],shape=[2,3],name='a')
    b=tf.constant([1.0,2.0,3.0,4.0,5.0,6.0],shape=[3,2],name='b')
c=tf.matmul(a,b)
sess=tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [22]:
#手工指派运行环镜GPU 电脑要自带GPU同时配置好相关环境
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# 新建 session with log_device_placement 并设置为 True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# 运行这个 op.
print (sess.run(c))

InvalidArgumentError: Cannot assign a device for operation 'MatMul_8': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: MatMul_8 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](a_7, b_7)]]

Caused by op 'MatMul_8', defined at:
  File "d:\software\python3.6\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\software\python3.6\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\software\python3.6\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\software\python3.6\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "d:\software\python3.6\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "d:\software\py

In [23]:
#使用多个GPU
# 新建一个 graph.
c = []
for d in ['/gpu:2', '/gpu:3']:
  with tf.device(d):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
    c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
  sum = tf.add_n(c)
# 新建session with log_device_placement并设置为True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# 运行这个op.
print (sess.run(sum))

InvalidArgumentError: Cannot assign a device for operation 'MatMul_8': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: MatMul_8 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](a_7, b_7)]]

Caused by op 'MatMul_8', defined at:
  File "d:\software\python3.6\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\software\python3.6\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\software\python3.6\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\software\python3.6\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "d:\software\python3.6\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "d:\software\py

In [ ]:
#第四个案例交互式的使用

In [33]:
import tensorflow as tf
#可以使用InteractiveSession代替Session
#使用Tensor.eval()和Operation.run()代替Session.run()
sess=tf.InteractiveSession()
x=tf.Variable([1.0,2.0])
a=tf.constant([3.0,3.0])
x.initializer.run()
sub=tf.subtract(x,a)
print(sub.eval())
##按照文档的说法是这是针对Ipython制定的一种方法，可以使用交互式的方法来进行模块化的处理，而不用每次都要要重新构建图才能运行，此案例将在后面碰到的时候说明或补充

[-2. -1.]


In [ ]:
#第五个案例，简单的计数器

In [5]:
#如果此时不能够正常运行，请重新打开文件，加载包，直接跳到此步骤运行
#state用来计数
state =tf.Variable(0,name="counter")
#one用来作为每次加一的数
one=tf.constant(1)
#将state加一
new_value=tf.add(state,one)
#将new_value赋值给state
update=tf.assign(state,new_value)#在下面sess.run调用之前不会进行操作
init_op=tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init_op)
    print(sess.run(state))
    for _ in range(3):#3决定了循环的次数
        sess.run(update)
        print(sess.run(state))
        

0
1
2
3


In [ ]:
#第六个案例 Fetch 取回操作 一次可以计算多个图，而不是原来的对两个图进行操作

In [4]:
input1=tf.constant(3.0)
input2=tf.constant(2.0)
input3=tf.constant(5.0)
intermed=tf.add(input2,input3)
mul=tf.multiply(input1,intermed)

with tf.Session() as sess:#如果没有sess则sess不会启动
    result=sess.run([mul,intermed])
    print(result)

[21.0, 7.0]


In [ ]:
#第七个案例 Feed 即我们可以使用feed_dict临时补充所没有的图，但是之前要有相应的占位符

In [11]:
input1=tf.placeholder(tf.float32)
input2=tf.placeholder(tf.float32)
out=tf.multiply(input1,input2)

with tf.Session() as sess:
    print(sess.run([out],feed_dict={input1:[7.],input2:[2.]}))

[array([14.], dtype=float32)]
